In [9]:
import pypdf
import chromadb
import urllib3
import accelerate
import sentence_transformers
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
%env TOKENIZERS_PARALLELISM=True
%env TF_CPP_MIN_LOG_LEVEL=3

env: TOKENIZERS_PARALLELISM=True
env: TF_CPP_MIN_LOG_LEVEL=3


## Extracting text data from PDF Files

In [11]:
# Create an object to load PDF file
loader = PyPDFLoader('../data/ArtigoDSA1.pdf')

type(loader)

langchain_community.document_loaders.pdf.PyPDFLoader

In [12]:
# Load the file PDF
pages = loader.load()

pages

[Document(metadata={'source': '../data/ArtigoDSA1.pdf', 'page': 0, 'page_label': '1'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \n \nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \npositivo.  \nPara dezenas de milhões de trabalhadores, não. \nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \ncontadores, os digitadores, os secretários executivos, procurando trabalho em uma nova \neconomia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker \nde Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram \nos pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. \nE não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de \ndólares e recairá mais fortemente em 

In [13]:
page = pages[2]

print("Page content: ", page.page_content[0:500])

Page content:  Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“


In [14]:
print("Metadata:", page.metadata)

Metadata: {'source': '../data/ArtigoDSA1.pdf', 'page': 2, 'page_label': '3'}


## Splitting Text Data in Chunks

**chunk_size = 1000**: Specifies that each resulting chunk of text will have a maximum of 1000 characters.

**chunk_overlap = 20**: Indicates that each chunk will have 20 characters of overlap with the next chunk. This means that the last 20 characters of a chunk will be repeated at the beginning of the next chunk.

What it is for:

This approach is useful in several situations where large texts need to be processed or analyzed, such as:

- Input for language models: Many LLMs have a limit of tokens that they can process in a single iteration. Dividing the text into smaller chunks ensures that the text is sent within the allowed limit.

- Data analysis and indexing: It is common in search engines and data processing pipelines, where dividing the text into smaller chunks makes it easier to index and retrieve information.

- Context maintenance: When processing involves long documents, this technique allows you to deal with them more efficiently, by dividing the parts without losing logic or cohesion.

In [15]:
# Create the chunk text separator
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)

In [18]:
# Applying the object and extracting the chunks (documents)
docs = splitter.split_documents(pages)

print("Total of Chunks (Documents):", len(docs))

print("Last Chunk Content (Document):", docs[6])

Total of Chunks (Documents): 7
Last Chunk Content (Document): page_content='Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“Seja Bom em Aprender”. Mantenha-se em modo constante de aprendizado. 
Equipe DSA 
www.datascienceacademy.com.br' metadata={'source': '../data/ArtigoDSA1.pdf', 'page': 2, 'page_label': '3'}
